In [44]:
import torch.nn as nn

In [45]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [74, 66, 43],
                   [91, 87, 65],
                   [88, 134, 59],
                   [101, 44, 37],
                   [68, 96, 71],
                   [73, 66, 44],
                   [92, 87, 64],
                   [87, 135, 57],
                   [103, 43, 36],
                   [68, 97, 70]],
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119],
                    [57, 69],
                    [80, 102],
                    [118, 132],
                    [21, 38],
                    [104, 118],
                    [57, 69],
                    [82, 100],
                    [118, 134],
                    [20, 38],
                    [102, 120]],
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

We are using 15 training examples to illustrate how to work with large datasets in small batches.

In [46]:
from torch.utils.data import TensorDataset

In [47]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [48]:
from torch.utils.data import DataLoader

In [49]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [50]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.4770, -0.5686, -0.1444],
        [ 0.3564, -0.2634,  0.4638]], requires_grad=True)
Parameter containing:
tensor([-0.2476, -0.3821], requires_grad=True)


In [51]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.4770, -0.5686, -0.1444],
         [ 0.3564, -0.2634,  0.4638]], requires_grad=True),
 Parameter containing:
 tensor([-0.2476, -0.3821], requires_grad=True)]

In [52]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ -9.7261,  27.9343],
        [-16.1114,  38.5586],
        [-43.3069,  22.2342],
        [ 18.6199,  41.8099],
        [-32.0211,  31.3923],
        [ -8.6805,  28.5541],
        [-15.6872,  39.2858],
        [-42.9742,  23.0544],
        [ 17.5743,  41.1901],
        [-32.6426,  31.4997],
        [ -9.3019,  28.6614],
        [-15.0658,  39.1784],
        [-43.7310,  21.5070],
        [ 19.2414,  41.7026],
        [-33.0667,  30.7725]], grad_fn=<AddmmBackward0>)

In [53]:
# Import nn.functional
import torch.nn.functional as F

In [54]:
# Define loss function
loss_fn = F.mse_loss

In [57]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(8371.4268, grad_fn=<MseLossBackward0>)


In [56]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [59]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):

    # Repeat for given number of epochs
    for epoch in range(num_epochs):

        # Train with batches of data
        for xb,yb in train_dl:

            # 1. Generate predictions
            pred = model(xb)

            # 2. Calculate loss
            loss = loss_fn(pred, yb)

            # 3. Compute gradients
            loss.backward()

            # 4. Update parameters using gradients
            opt.step()

            # 5. Reset the gradients to zero
            opt.zero_grad()

        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [68]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 1.3911
Epoch [20/100], Loss: 1.2479
Epoch [30/100], Loss: 0.7032
Epoch [40/100], Loss: 0.9518
Epoch [50/100], Loss: 0.9321
Epoch [60/100], Loss: 0.9593
Epoch [70/100], Loss: 0.7527
Epoch [80/100], Loss: 1.4393
Epoch [90/100], Loss: 0.8657
Epoch [100/100], Loss: 0.9061


In [62]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 56.9232,  70.3373],
        [ 81.9558, 100.5484],
        [118.4240, 132.8787],
        [ 20.8834,  37.7866],
        [101.6390, 118.4222],
        [ 55.6763,  69.2614],
        [ 81.7948, 100.6409],
        [118.7114, 133.4831],
        [ 22.1303,  38.8625],
        [102.7248, 119.5906],
        [ 56.7621,  70.4298],
        [ 80.7090,  99.4725],
        [118.5850, 132.7862],
        [ 19.7976,  36.6182],
        [102.8859, 119.4981]], grad_fn=<AddmmBackward0>)

In [63]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [67]:
model(torch.tensor([[91,  88,  64.]]))

tensor([[ 82.0390, 100.6067]], grad_fn=<AddmmBackward0>)